In [ ]:
import numpy as np
import tensorflow as tf
import time
import os
import datetime
import matplotlib.pyplot as plt

In [ ]:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
class_names = [
    'T-shirt/top', 
    'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
#fashion_mnist = tf.keras.datasets.fashion_mnist
#(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
plt.imshow(x_train[0], cmap='gray', interpolation='none')

In [ ]:
plt.figure(figsize=(15,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_train[i], cmap='gray', interpolation='none')
    plt.title(f"Class {y_train[i]} {class_names[y_train[i]]}")

## Logistic Regresion Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(10, activation='softmax'),
])  

## Deep Learning Model

![alt text](figure.png "Title")

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

## Entrenamiento

In [ ]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

logdir = os.path.join("tutorial_4", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=10,
          batch_size=100,
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

# Evaluamos el resultado

In [ ]:
score = model.evaluate(x_train, y_train,
                      verbose=0)
print('Train accuracy:', score[1])

In [ ]:
score = model.evaluate(x_test, y_test,
                      verbose=0)
print('Test accuracy:', score[1])

https://github.com/zalandoresearch/fashion-mnist

In [ ]:
model

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
y_prob = model.predict(x_test) 
predicted_classes = y_prob.argmax(axis=-1)

# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

___

In [ ]:
incorrect = 7

In [ ]:
prob = model.predict(x_test[incorrect:incorrect+1])

In [ ]:
plt.imshow(x_test[incorrect], cmap='gray', interpolation='none')

In [ ]:
prob

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.bar(height = prob[0], x=['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'])

In [ ]:
plt.figure(figsize=(20,10))
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    #plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_test[correct]))
    plt.title(f"Predicted Class {predicted_classes[correct]} {class_names[predicted_classes[correct]]}  True Class: {y_test[correct]} {class_names[y_test[correct]]}")

In [ ]:
plt.figure(figsize=(20,10))
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(f"Predicted Class {predicted_classes[incorrect]} {class_names[predicted_classes[incorrect]]}  True Class: {y_test[incorrect]} {class_names[y_test[incorrect]]}")

In [ ]:
test_pred_raw = model.predict(x_test)
test_pred = np.argmax(test_pred_raw, axis=1)

import sklearn.metrics
import itertools

def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Compute the labels from the normalized confusion matrix.
  labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure


# Calculate the confusion matrix.
cm = sklearn.metrics.confusion_matrix(y_test, test_pred)
# Log the confusion matrix as an image summary.
figure = plot_confusion_matrix(cm, class_names=class_names)